# Roteamento

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(model='gpt-4o-mini')

prompt = ChatPromptTemplate.from_template("""Você é um professor de matemática de ensino fundamental
capaz de dar respostas muito detalhadas e didáticas. Responda a seguinte pergunta de um aluno:
Pergunta: {pergunta}""")
chain_matematica = prompt | model

prompt = ChatPromptTemplate.from_template("""Você é um professor de física de ensino fundamental
capaz de dar respostas muito detalhadas e didáticas. Responda a seguinte pergunta de um aluno:
Pergunta: {pergunta}""")
chain_fisica = prompt | model

prompt = ChatPromptTemplate.from_template("""Você é um professor de história de ensino fundamental
capaz de dar respostas muito detalhadas e didáticas. Responda a seguinte pergunta de um aluno:
Pergunta: {pergunta}""")
chain_historia = prompt | model

prompt = ChatPromptTemplate.from_template("""{pergunta}""")
chain_generica = prompt | model

In [3]:
from pydantic import BaseModel, Field

prompt = ChatPromptTemplate.from_template("Você deve categorizar a seguinte pergunta: {pergunta}")

class Categorizador(BaseModel):
    """Categoriza as perguntas de alunos do ensino fundamental"""
    area_conhecimento: str = Field(description="A área de conhecimento da pergunta feita pelo aluno. \
    Deve ser 'física', 'matemática' ou 'história'. Caso não se encaixe em nenhuma delas, retorne 'outra'")

model_estruturado = prompt | model.with_structured_output(Categorizador)
model_estruturado.invoke({"pergunta": "Quando foi a independência dos Estados Unidos?"})

Categorizador(area_conhecimento='história')

## Criando estrutura de roteamento

O RunnablePassthrough é um tipo de Runnable que simplesmente passa a entrada recebida como saída, podendo também adicionar chaves adicionais ao resultado. Ele se comporta quase como uma função de identidade, mas é útil em cenários onde você deseja incluir lógica adicional ou manipular a entrada de alguma forma antes de passar adiante.

In [4]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough().assign(categoria=model_estruturado)
chain.invoke({'pergunta': 'Quando foi a independência dos Estados Unidos?'})

{'pergunta': 'Quando foi a independência dos Estados Unidos?',
 'categoria': Categorizador(area_conhecimento='história')}

In [5]:
def route(input):
    if input['categoria'].area_conhecimento == 'matemática':
        return chain_matematica
    if input['categoria'].area_conhecimento == 'física':
        return chain_fisica
    if input['categoria'].area_conhecimento == 'história':
        return chain_historia
    return chain_generica

In [6]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough().assign(categoria=model_estruturado) | route
chain.invoke({'pergunta': 'Quando foi a independência dos Estados Unidos?'})

AIMessage(content='A independência dos Estados Unidos foi declarada no dia 4 de julho de 1776. Vamos entender melhor o contexto e a importância desse evento na história.\n\n**Contexto Histórico:**\n\nAntes de se tornarem independentes, os Estados Unidos eram 13 colônias britânicas que se localizavam na costa atlântica da América do Norte. Essas colônias eram controladas pela Grã-Bretanha, e os colonos enfrentavam uma série de impostos e regulamentações que consideravam injustos. Por exemplo, o britânico governo introduziu o "Stamp Act" em 1765 e o "Tea Act" em 1773, que levantaram uma série de protestos. Os colonos sentiam que não podiam ser taxados sem representação, o que levou ao famoso slogan "Sem tributação sem representação".\n\n**A Revolução Americana:**\n\nAs tensões entre os colonos e a Grã-Bretanha aumentaram ao longo dos anos e culminaram em uma série de eventos, incluindo o Boston Tea Party em que colonos, disfarçados de indígenas americanos, jogaram chá no porto de Boston 